In [ ]:
# delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 2.3MB 33.9MB/s 
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp37-none-any.whl size=2976 sha256=db079b1d858d343e6aacdb4dfd11e7ce464d674558f1118eb88ef6d1c1b1dc8d
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Ex4files/"

from Bio.PDB import *
import numpy as np
import re
import os
import pickle
from tqdm import tqdm

/content/drive/MyDrive/Ex4files


In [ ]:
W_RPattern = re.compile(r'W\wR')
Y_CPattern = re.compile(r'(Y\wC)|(YY\w)')
WG_GPattern = re.compile(r'(WG\wG)|(W\wQG)|(\wGQG)|(\w\wQG)|(WG\w\w)|(W\w\wG)|(W\wQ\w)')
CDR_MAX_LENGTH = 32
AA_DICT = {"A": 0, "C": 1, "D": 2, "E": 3, "F": 4, "G": 5, "H": 6, "I": 7, "K": 8, "L": 9, "M": 10, "N": 11,
           "P": 12, "Q": 13, "R": 14, "S": 15, "T": 16, "W": 17, "Y": 18, "V": 19, "-": 20, "X": 20}


In [ ]:
def find_cdr3(sequence):
    """
    find the start and end indexes of the CDR3 in the given nanobody sequence
    :param sequence: String
    :return: [start, end]  (ints)
    """
    left_area = sequence[90:105]
    la_i = -1
    left_cdr = -1
    Y_C = Y_CPattern.search(left_area)
    if Y_C != None:
        # if we found 'YXR', find its index
        la_i = Y_C.start(0)+2
    else:
        la_i = left_area.find('C')  # didn't find 'YXC', look for 'C'

    if la_i >= 0:
        left_cdr = la_i + 90 + 3
    n = len(sequence) - 1
    n1 = n - 14
    subtract_amount = 1
    right_area = sequence[n1:n - 4]
    ra_i = -1
    right_cdr = -1
    WG_G = WG_GPattern.search(right_area)
    if WG_G != None:
        ra_i = WG_G.start(0)

    if ra_i >= 0:
        right_cdr = ra_i + n1 - subtract_amount + 1  # CDR3 ends at 'W' - 1 (or 'Q' - 3) (add n-14 to put it back in the full sequence)
    # check
    if left_cdr == -1 and right_cdr == -1:
        left_cdr = n - 21
        right_cdr = n - 10
    elif left_cdr == -1:
        left_cdr = right_cdr - 11
    elif right_cdr == -1:
        if left_cdr + 11 <= n:
            right_cdr = left_cdr + 11
        else:
            right_cdr = n
    if left_cdr > right_cdr:
        left_cdr = n - 1
        right_cdr = n
    return [left_cdr,right_cdr]



In [ ]:
def printPretty():
  print("hello")

In [ ]:
def get_seq_aa(chain):
    """
    returns the sequence (String) and a list of all the aa residue objects of the given nanobody chain.
    :param chain: BioPython chain object
    :return: sequence, [aa objects]
    """
    aa_residues = []
    seq = ""

    for residue in chain.get_residues():
        aa = residue.get_resname()
        if not is_aa(aa) or not residue.has_id('CA'):
            continue
        elif aa == "UNK":
            seq += "X"
            aa_residues.append(residue)
        else:
            seq += Polypeptide.three_to_one(residue.get_resname())
            aa_residues.append(residue)

    return seq, aa_residues

In [ ]:
def add_padding(matrix):
    """
    adds zero padding to a given matrix (even padding in each direction). matrix has to be of size n*n*m where
    n <= CDR_MAX_LENGTH
    :param matrix: numpy array to pad
    :return: the padded numpy array, size 32* 32*m
    """
    smaller_pad = (CDR_MAX_LENGTH - len(matrix)) // 2
    bigger_pad = CDR_MAX_LENGTH - len(matrix) - smaller_pad

    pad_x_y = (smaller_pad, bigger_pad)
    pad_z = (0,0)

    return np.pad(matrix, (pad_x_y, pad_x_y, pad_z))

In [ ]:
def remove_padding(matrix, cdr_length):
    """
    removes the zero padding added to a numpy array using add_padding(), cdr_length is the size of the matrix before
    calling add_padding().
    :param matrix: numpy array of size (CDR_MAX_LENGTH * CDR_MAX_LENGTH * m)
    :param cdr_length: the original size of the matrix (the cdr3 length), int
    :return: a numpy array of size (cdr_length * cdr_length * m)
    """
    pad = (CDR_MAX_LENGTH - (cdr_length)) // 2
    return matrix[pad:pad + cdr_length, pad:pad + cdr_length]


In [ ]:
def get_dist(residues):
    """
    calculates all the pairwise distances of the aa residues in the given residues list.
    divides all the distances by a factor of 10.
    :param residues: list of aa residue object
    :return: numpy array of size (len(residues) * len(residues) * 1)
    """
    mat_size = len(residues)
    dist = np.zeros((mat_size, mat_size))

    for i in range(mat_size):
        for j in range(mat_size):
            if i == j:
                continue
            c1 = 'CB'
            c2 = 'CB'
            if 'CB' not in residues[i]:  # GLY
                c1 = 'CA'
            if 'CB' not in residues[j]:  # GLY
                c2 = 'CA'
            dist[i][j] = (residues[i][c1] - residues[j][c2])

    dist = dist / 10
    return dist.reshape((mat_size, mat_size, 1))


In [ ]:
def get_theta(residues):
    """
    calculates all the pairwise theta angles of the aa residues in the given residues list.
    returns the pairwise angle in their (sin, cos) representation.
    :param residues: list of aa residue object
    :return: numpy array of size (len(residues) * len(residues) * 2)
    """

    mat_size = len(residues)
    cos_theta = np.zeros((mat_size, mat_size))
    sin_theta = np.zeros((mat_size, mat_size))

    for i in range(len(residues)):
        for j in range(len(residues)):
            if i == j:
                continue
            if not residues[i].has_id('CB') or not residues[j].has_id('CB'):  # GLY OR UNK OR MISSING CB
                continue

            angle = calc_dihedral(residues[i]["N"].get_vector(), residues[i]["CA"].get_vector(),
                                  residues[i]["CB"].get_vector(), residues[j]["CB"].get_vector())
            cos_theta[i][j] = np.cos(angle)
            sin_theta[i][j] = np.sin(angle)

    return np.dstack([sin_theta, cos_theta])

In [ ]:
def get_phi(residues):
    """
    calculates all the pairwise phi angles of the aa residues in the given residues list.
    returns the pairwise angle in their (sin, cos) representation.
    :param residues: list of aa residue object
    :return: numpy array of size (len(residues) * len(residues) * 2)
    """
    mat_size = len(residues)
    cos_phi = np.zeros((mat_size, mat_size))
    sin_phi = np.zeros((mat_size, mat_size))

    for i in range(len(residues)):
        for j in range(len(residues)):
            if i == j:
                continue
            if not residues[i].has_id('CB') or not residues[j].has_id('CB'):  # GLY OR UNK OR MISSING CB
                continue
            angle = calc_angle(residues[i]["CA"].get_vector(),
                               residues[i]["CB"].get_vector(),
                               residues[j]["CB"].get_vector())
            cos_phi[i][j] = np.cos(angle)
            sin_phi[i][j] = np.sin(angle)

    return np.dstack([sin_phi, cos_phi])

In [ ]:
def get_omega(residues):
    """
    calculates all the pairwise omega angles of the aa residues in the given residues list.
    returns the pairwise angle in their (sin, cos) representation.
    :param residues: list of aa residue object
    :return: numpy array of size (len(residues) * len(residues) * 2)
    """
    mat_size = len(residues)
    cos_phi = np.zeros((mat_size, mat_size))
    sin_phi = np.zeros((mat_size, mat_size))

    for i in range(len(residues)):
        for j in range(len(residues)):
            if i == j:
                continue
            if not residues[i].has_id('CB') or not residues[j].has_id('CB'):  # GLY OR UNK OR MISSING CB
                continue

            angle = calc_dihedral(residues[i]["CA"].get_vector(), residues[i]["CB"].get_vector(),
                                  residues[j]["CB"].get_vector(), residues[j]["CA"].get_vector())
            cos_phi[i][j] = np.cos(angle)
            sin_phi[i][j] = np.sin(angle)

    return np.dstack([sin_phi, cos_phi])

In [ ]:
def generate_input(pdb_file):
    """
    receives a pdb file and returns its CDR3 sequence in a one-hot encoding matrix (each row is an aa in the sequence, and
    each column represents a different aa out of the 20 aa + 1 special column).
    :param pdb_file: path to a pdb file (nanobody, heavy chain has id 'H')
    :return: numpy array of size (CDR_MAX_LENGTH * 21)
    """
    # load model
    model = PDBParser().get_structure(pdb_file, pdb_file)[0]["H"]

    # get seq and aa residues
    seq, aa_residues = get_seq_aa(model)

    # find cdr3 start and end
    [cdr_start, cdr_end] = find_cdr3(seq)
    cdr_len = (cdr_end + 1 - cdr_start)

    cdr_up_pad = (CDR_MAX_LENGTH - cdr_len) // 2
    cdr_down_pad = CDR_MAX_LENGTH - cdr_up_pad - cdr_len

    # pad the sequence with '-'
    seq_cdr = cdr_up_pad * "-" + seq[cdr_start:cdr_end + 1] + cdr_down_pad * "-"

    # turn in to one-hot encoding matrix
    cdr_matrix = np.zeros((CDR_MAX_LENGTH, 21))
    for i in range(CDR_MAX_LENGTH):
        cdr_matrix[i][AA_DICT[seq_cdr[i]]] = 1

    return cdr_matrix

In [ ]:
def generate_label(pdb_file):
    """
    receives a pdb file and returns its pairwise distances and pairwise angles (omega, theta, phi).
    :param pdb_file: path to a pdb file (nanobody, heavy chain has id 'H')
    :return: 4 numpy arrays, the first one with size (CDR_MAX_LENGTH * CDR_MAX_LENGTH * 1) and the other three with
    size (CDR_MAX_LENGTH * CDR_MAX_LENGTH * 2).
    """
    # load model
    model = PDBParser().get_structure(pdb_file, pdb_file)[0]["H"]

    # get seq and aa residues
    seq, aa_residues = get_seq_aa(model)

    # find cdr3 start and end
    [cdr_start, cdr_end] = find_cdr3(seq)


    # get the cdr3 residues
    aa_residues = aa_residues[cdr_start:cdr_end + 1]

    # get angles and distance matrices
    theta = get_theta(aa_residues)
    dist = get_dist(aa_residues)
    phi = get_phi(aa_residues)
    omega = get_omega(aa_residues)
   


    return add_padding(dist), add_padding(omega), add_padding(theta), add_padding(phi)


In [ ]:
if __name__ == '__main__':
    
   #  you can make all the data for the network in this section. use picke dump to save all the 5 matrices.
   # this way you won't have to generate them each time you train a newtork.
   # you can save the matrices to your drive and load them in your google colab file later.


    input_matrix = []
    dist_matrix = []
    omega_matrix = []
    theta_matrix = []
    phi_matrix = []

    dist_ref = None
    input_ref = None

    c
    
    for pdb in tqdm(os.listdir(data_path)):
      if pdb.endswith(".pdb"):
        if(pdb == "ref.pdb"):
          dist_ref, omega, theta, phi = generate_label(os.path.join(data_path, pdb))
          input_ref = generate_input(os.path.join(data_path, pdb))
        else:
          dist, omega, theta, phi = generate_label(os.path.join(data_path, pdb))
          input_matrix.append(generate_input(os.path.join(data_path, pdb)))
          dist_matrix.append(dist)
          omega_matrix.append(omega)
          theta_matrix.append(theta)
          phi_matrix.append(phi)

    save_path = "/content/drive/MyDrive/Ex4files/Ex4Output"  # TODO: change path if needed

    pickle.dump(np.array(dist_matrix), open(os.path.join(save_path , "train_dist.pkl"), "wb"))
    pickle.dump(np.array(omega_matrix), open(os.path.join(save_path , "train_omega.pkl"), "wb"))
    pickle.dump(np.array(theta_matrix), open(os.path.join(save_path , "train_theta.pkl"), "wb"))
    pickle.dump(np.array(phi_matrix), open(os.path.join(save_path , "train_phi.pkl"), "wb"))
    pickle.dump(np.array(dist_ref), open(os.path.join(save_path , "ref_dist.pkl"), "wb"))
    pickle.dump(np.array(input_ref), open(os.path.join(save_path , "ref_input.pkl"), "wb"))


    pickle.dump(np.array(input_matrix), open(os.path.join(save_path , "train_input.pkl"), "wb"))

    print("Number of samples: {}".format(len(input_matrix)))



  1%|          | 11/2150 [00:06<21:57,  1.62it/s]/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 1867.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 1867.
  PDBConstructionWarning,
  5%|▍         | 97/2150 [01:01<21:02,  1.63it/s]/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 1857.
  PDBConstructionWarning,
/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 1857.
  PDBConstructionWarning,
  8%|▊         | 166/2150 [01:41<17:50,  1.85it/s]/usr/local/lib/python3.7/dist-packages/Bio/PDB/StructureBuilder.py:92: PDBConstructionWarning: WARNING: Chain L is discontinuous at line 1726.
  PDBConstructionWarning,
/usr/lo

Number of samples: 2141
